In [ ]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Ensures that all dataframes are displayed on one line instead of breaking columns across multiple lines
pd.set_option('display.expand_frame_repr', False)

sparql = SPARQLWrapper('https://vocab.getty.edu/sparql')
sparql.setReturnFormat(JSON)

In [ ]:
def make_query(query):
    sparql.setQuery(query)
    results = sparql.queryAndConvert()
    return pd.json_normalize(results['results']['bindings'])

## Simple Structure of a Query

In [ ]:
query = '''
# Can be replaced with * to select all variables
SELECT ?subject ?predicate ?object
WHERE {
    ?subject ?predicate ?object .
}

OFFSET 10 LIMIT 10
'''

make_query(query)

## Prefixes

In [ ]:
query = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT *
WHERE {
    ?subject rdfs:label ?object .
}
LIMIT 10
'''

make_query(query)

## Shorthand Notation

In [ ]:
query = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX xl: <http://www.w3.org/2008/05/skos-xl#>

SELECT ?collaboratorName ?parentName
WHERE {
    ?person rdfs:label "Vincent van Gogh" .
    ?person gvp:ulan1303_collaborated_with ?collaborator .
    ?person gvp:ulan1511_child_of ?parent .

    ?collaborator gvp:prefLabelGVP ?collaboratorPrefLabel .
    ?collaboratorPrefLabel xl:literalForm ?collaboratorName .

    ?parent gvp:prefLabelGVP ?parentPrefLabel .
    ?parentPrefLabel xl:literalForm ?parentName .
}
LIMIT 10
'''

make_query(query)

In [ ]:
query = '''
SELECT ?collaborator ?parent
WHERE {
    ?person rdfs:label "Vincent van Gogh" ;
            gvp:ulan1303_collaborated_with [ gvp:prefLabelGVP [ xl:literalForm ?collaborator ]] ;
            gvp:ulan1511_child_of [ gvp:prefLabelGVP [ xl:literalForm ?parent ]] .
}
LIMIT 10
'''

make_query(query)

## Optional Clauses

In [ ]:
query = '''
SELECT ?artist ?artistName ?niecesNephews
WHERE {
    ?artist gvp:broaderInstantial ulan:500000002 ;
            gvp:prefLabelGVP [ xl:literalForm ?artistName ] .

    OPTIONAL {
        ?artist gvp:ulan1532_uncle-aunt_of ?niecesNephews .
    }
    
}
OFFSET 940 LIMIT 10
'''

make_query(query)

## Filters

In [ ]:
query = '''
SELECT ?artist ?artistName ?birthYear ?deathYear
WHERE {
    ?artist gvp:broaderInstantial ulan:500000002 ;
            gvp:prefLabelGVP [ xl:literalForm ?artistName ] ;
            foaf:focus [ gvp:biographyPreferred [ gvp:estStart ?birthYear ; gvp:estEnd ?deathYear ]] .

    FILTER(YEAR(?birthYear) > 1800) # && YEAR(?birthYear) < 2024
    
}
ORDER BY ASC(?birthYear)
LIMIT 10
'''

make_query(query)

## Values

In [ ]:
query = '''
SELECT ?name ?person ?collaborator ?parent
WHERE {
    VALUES (?name) { ("Vincent van Gogh") ("Paul Klee") ("John Singer Sargent") }

    ?person rdfs:label ?name .

    OPTIONAL {
        ?person gvp:ulan1303_collaborated_with [ gvp:prefLabelGVP [ xl:literalForm ?collaborator ]] ;
                gvp:ulan1511_child_of [ gvp:prefLabelGVP [ xl:literalForm ?parent ]] .
    }
}
'''

make_query(query)

## Group by Clauses

In [ ]:
query = '''
SELECT ?birthYear (COUNT(?artist) AS ?count)
WHERE {
    ?artist gvp:broaderInstantial ulan:500000002 ;
            gvp:prefLabelGVP [ xl:literalForm ?artistName ] ;
            foaf:focus [ gvp:biographyPreferred [ gvp:estStart ?birthYear ; gvp:estEnd ?deathYear ]] .
}
GROUP BY ?birthYear
ORDER BY DESC(?count)
LIMIT 10
'''

make_query(query)

## Federated Queries

In [ ]:
query = '''
PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>

SELECT ?subject ?predicate ?object
WHERE {
    SERVICE <http://dbpedia.org/sparql> {
        SELECT ?subject ?predicate ?object
        WHERE {
            ?subject ?predicate ?object .
        }
        LIMIT 1
    }
}
'''

make_query(query)